Adding post-processed variables for Facebook 2022 and create the "var table" (fb_adid_var.csv).

Merging output from these repos: [entity_linking_2022](https://github.com/Wesleyan-Media-Project/entity_linking_2022), [ABSA](https://github.com/Wesleyan-Media-Project/ABSA), [race_of_focus](https://github.com/Wesleyan-Media-Project/race_of_focus),  [party_classifier](https://github.com/Wesleyan-Media-Project/party_classifier), [ad_tone](https://github.com/Wesleyan-Media-Project/ad_tone), [ad_goal_classifier](https://github.com/Wesleyan-Media-Project/ad_goal_classifier), [party_classifier_pdid](https://github.com/Wesleyan-Media-Project/party_classifier_pdid), and [issue_classifier](https://github.com/Wesleyan-Media-Project/issue_classifier)

### Output table:
+ fb_2022_adid_var.csv

In [1]:
import pandas as pd
import numpy as np
import os

### Load fb2022 var1

In [2]:
'''
Load the var1 table produced from 

data-post-production/01-merge-results/01_merge_preprocessed_multimedia_results.ipynb
'''

d = pd.read_csv("../../fb_2022_adid_var1.csv.gz")

In [3]:
d.shape

(377721, 15)

In [4]:
d.head(2)

,ad_id,page_id,ad_creation_time,ad_delivery_start_time,ad_delivery_stop_time,spend,impressions,region_distribution,demographic_distribution,publisher_platforms,potential_reach,pd_id,wmp_media_type,aws_face_vid,aws_face_img
0,x_1100321017564452,260713150211,2022-08-12,2022-08-13,2023-01-06,"{""lower_bound"":""100"",""upper_bound"":""199""}","{""lower_bound"":""2000"",""upper_bound"":""2999""}","[{""percentage"":""0.005223"",""region"":""Mississipp...","[{""percentage"":""0.005834"",""age"":""18-24"",""gende...","[""facebook"",""instagram""]","{""lower_bound"":""1000001""}",pd-260713150211-4,image,NaN,NaN
1,x_624349849305317,154561574598964,2022-08-29,2022-08-30,2023-01-08,"{""lower_bound"":""1500"",""upper_bound"":""1999""}","{""lower_bound"":""175000"",""upper_bound"":""199999""}","[{""percentage"":""1.0E-5"",""region"":""Comunidad Va...","[{""percentage"":""0.000698"",""age"":""25-34"",""gende...",facebook,"{""lower_bound"":""1000001""}",pd-154561574598964-4,image,NaN,WMPID5215


In [5]:
d['aws_face_vid'].unique()

array([nan, 'WMPID5281', 'WMPID1289', ..., 'WMPID873,WMPID84',
       'WMPID1052,WMPID84', 'WMPID406,WMPID1289,WMPID1533'], dtype=object)

### Add race of focus

In [6]:
rof = pd.read_csv("../../../race_of_focus/data/race_of_focus_fb2022.csv")

In [7]:
rof.shape

(377721, 4)

In [8]:
rof.head(2)

,ad_id,sub_bucket,race_of_focus,race_of_focus_region_pct
0,x_1100321017564452,3.3,No race of focus,NaN
1,x_624349849305317,3.3,No race of focus,NaN


In [9]:
d = d.merge(rof, on='ad_id', how='left')

### Add tone - constructed

In [10]:
tone_c=pd.read_csv("../../../ad_tone/data/ad_tone_constructed_fb2022.csv.gz")

In [11]:
tone_c.shape

(91221, 2)

In [12]:
tone_c.head(2)

,ad_id,ad_tone_constructed
0,x_337588521869565,Promote
1,x_340739824879297,Promote


In [13]:
d = d.merge(tone_c, on='ad_id', how='left')

### Add tone - mentioned

In [14]:
tone_m=pd.read_csv("../../../ad_tone/data/ad_tone_mentionbased_fb2022.csv")

In [15]:
tone_m.shape

(46543, 2)

In [16]:
tone_m.head(2)

,ad_id,ad_tone
0,x_337588521869565,Promote
1,x_340739824879297,Promote


In [17]:
tone_m=tone_m.rename(columns={'ad_tone':'ad_tone_mentionbased'})

In [18]:
d = d.merge(tone_m, on='ad_id', how='left')

### Add goal

In [19]:
goal = pd.read_csv("../../../ad_goal_classifier/data/ad_goal_rf_fb2022.csv.gz")

In [20]:
goal.shape

(377721, 20)

In [21]:
goal.head(2)

,ad_id,goal_DONATE_prediction,goal_DONATE_predicted_prob,goal_CONTACT_prediction,goal_CONTACT_predicted_prob,goal_PURCHASE_prediction,goal_PURCHASE_predicted_prob,goal_GOTV_prediction,goal_GOTV_predicted_prob,goal_EVENT_prediction,goal_EVENT_predicted_prob,goal_POLL_prediction,goal_POLL_predicted_prob,goal_GATHERINFO_prediction,goal_GATHERINFO_predicted_prob,goal_LEARNMORE_prediction,goal_LEARNMORE_predicted_prob,goal_PRIMARY_PERSUADE_prediction,goal_PRIMARY_PERSUADE_predicted_prob,goal_highest_prob
0,x_1100321017564452,0,0.002697,0,0.009425,0,0.012842,0,0.027895,0,0.003091,0,0.021264,0,0.111034,1,0.700723,1,0.620700,LEARNMORE
1,x_624349849305317,0,0.007991,0,0.007874,0,0.005900,0,0.025523,0,0.005360,0,0.008017,0,0.030201,1,0.588740,1,0.789359,PRIMARY_PERSUADE


In [22]:
d = d.merge(goal, on='ad_id', how='left')

### Add party classifier trained at the pdid level

In [23]:
party_pdid = pd.read_csv("../../../party_classifier_pdid/party_all_clf_pdid_fb_2022.csv")

In [24]:
party_pdid.head(2)

,pd_id,party_all_clf_pdid
0,pd-1000253160054572-3,DEM
1,pd-100053898949-2,REP


In [25]:
d = d.merge(party_pdid, on='pd_id', how='left')

### Add party classifier at the ad level

In [26]:
party_ad = pd.read_csv("../../../party_classifier (Ad Level)/data/facebook/party_predictions_fb_2022.csv.gz")

# This one was not in the 2020 so I did not include it
#party_ad2 = pd.read_csv("../../../party_classifier (Ad Level)/data/facebook/party_predictions_fb_2022_both_model.csv.gz")

In [27]:
party_ad.head(2)

,ad_id,prob_dem,prob_other,prob_rep,predicted_party_all,predicted_party_all_majvote
0,x_1677811605924978,0.519075,0.0366,0.444326,DEM,DEM
1,x_5994752437219087,0.519075,0.0366,0.444326,DEM,DEM


In [28]:
party_ad.rename(columns={'predicted_party_all': 'party_all_clf_adid', 
                         'predicted_party_all_majvote': 'party_all_clf_adid_agg'}, inplace=True)
party_ad.head(2)

,ad_id,prob_dem,prob_other,prob_rep,party_all_clf_adid,party_all_clf_adid_agg
0,x_1677811605924978,0.519075,0.0366,0.444326,DEM,DEM
1,x_5994752437219087,0.519075,0.0366,0.444326,DEM,DEM


In [29]:
d = d.merge(party_ad, on='ad_id', how='left')

### WMP party_all

In [30]:
wmp = pd.read_csv("../../../datasets/wmp_entity_files/Facebook/2022/wmp_fb_2022_entities_v120122.csv", usecols=['pd_id', 'party_all'])

In [31]:
wmp = wmp.replace('MISSING', np.nan)

In [32]:
wmp.party_all.value_counts(dropna=False)

party_all
NaN      11727
DEM       3146
REP       3050
OTHER      303
Name: count, dtype: int64

In [33]:
d = d.merge(wmp, on='pd_id', how='left')

### Add detected entities

In [34]:
el = pd.read_csv("../../../entity_linking_2022/facebook/data/detected_entities_fb22.csv.gz")

In [35]:
el.shape

(168144, 2)

In [36]:
el.tail()

,ad_id,detected_entities
168139,x_999705858093151,WMPID1289|WMPID1289|WMPID1594|WMPID1289|WMPID1...
168140,x_999707127464521,WMPID4545|WMPID4545|WMPID3771|WMPID4545|WMPID4...
168141,x_999743237373888,WMPID1290|WMPID1290
168142,x_999777531419792,WMPID1305|WMPID1305
168143,x_999880071405435,WMPID5308|WMPID5308


In [37]:
el['detected_entities'] = [i.replace('|', ',') for i in el['detected_entities']]

In [38]:
d = d.merge(el, on='ad_id', how='left')

In [39]:
print(d.shape)
print(len(d.ad_id.unique()))

(377721, 47)
377721


### Add detected entities federal

In [40]:
cand = pd.read_csv('../../../datasets/candidates/wmpcand_120223_wmpid.csv')

In [41]:
cand.shape

(4295, 28)

In [42]:
cand.head(2)

,cand_id,wmpid,genelect_cd,CurrCand,cand_name,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name,first_name,last_name,st_dist
0,H0AL01055,WMPID21,1,Y,"CARL, JERRY LEE, JR",H,AL,1.0,REP,INCUMBENT,...,N,White,W,NaN,N,0,Jerry Carl,Jerry,Carl,AL1
1,H0AL02202,WMPID24,1,Y,"HARVEY-HALL, PHYLLIS",H,AL,2.0,DEM,CHALLENGER,...,n,Black,B,NaN,n,0,Phyllis Harvey-Hall,Phyllis,Harvey-Hall,AL2


In [43]:
cand['cand_office'].value_counts(dropna=False)

cand_office
H    3612
S     683
Name: count, dtype: int64

In [44]:
candlist = cand['wmpid'].tolist()

In [45]:
'WMPID4536' in candlist

True

In [46]:
d['detected_entities_federal'] = d['detected_entities'].apply(lambda x: ','.join(list(set([item.strip() for item in str(x).split(',') if item.strip() in candlist]))))

In [47]:
d['detected_entities_federal']= d['detected_entities_federal'].replace('', np.NaN)

### Add ABSA (Sentiment)

In [57]:
sent = pd.read_csv("../../../ABSA/data/fb2022_ABSA_pred.csv.gz")

In [58]:
sent.head(2)

,text_detected_entities,text_start,text_end,ad_id,field,predicted_sentiment
0,WMPID3138,33,39,x_2245155718992968,disclaimer,1
1,WMPID3138,33,39,x_429314142607623,disclaimer,1


In [62]:
sent.shape

(755444, 6)

In [69]:
grouped = sent.groupby(['ad_id', 'text_detected_entities'])
average_scores = grouped['predicted_sentiment'].sum().reset_index()
repetition_count = grouped.size().reset_index(name='number_of_mentions')
sent2 = pd.merge(average_scores, repetition_count, on=['ad_id', 'text_detected_entities'])


In [70]:
sent2.rename(columns={'text_detected_entities': 'ABSA_detected_entities', 
                      'predicted_sentiment': 'ABSA_predicted_sentiment_agg',
                      'number_of_mentions': 'ABSA_number_of_mentions'}, inplace=True)


In [71]:
sent2

,ad_id,ABSA_detected_entities,ABSA_predicted_sentiment_agg,ABSA_number_of_mentions
0,x_1000032110674951,WMPID2029,-1,1
1,x_1000072604727964,WMPID1289,-1,1
2,x_1000263194705315,WMPID1504,1,1
3,x_1000277107301672,WMPID5277,-19,23
4,x_1000277107301672,WMPID5326,-1,1
...,...,...,...,...
272598,x_999707127464521,WMPID3771,-1,1
272599,x_999707127464521,WMPID4545,-1,5
272600,x_999743237373888,WMPID1290,-2,2
272601,x_999777531419792,WMPID1305,2,2


In [73]:
agg_sent = sent2.groupby('ad_id').agg({'ABSA_number_of_mentions': lambda x: ','.join(map(str, x)),
                                       'ABSA_detected_entities': lambda x: ','.join(x),
                                       'ABSA_predicted_sentiment_agg': lambda x: ','.join(map(str, x))}).reset_index()

In [75]:
d = d.merge(agg_sent, on='ad_id', how='left')

### Add Issue Classifier

In [77]:
issue = pd.read_csv("../../../issue_classifier/fb_2022/data/fb2022_output_processed.csv")

In [78]:
issue.head(2)

,ad_id,issue_field,issue_class
0,x_1000009254731384,google_asr_text|aws_ocr_text_vid|ad_creative_body,"ISSUE10,ISSUE11,ISSUE56|ISSUE10,ISSUE56|ISSUE10"
1,x_1000018871393117,ad_creative_link_description|page_name|ad_crea...,"ISSUE12|ISSUE55|ISSUE55,ISSUE102|ISSUE16"


In [79]:
d = d.merge(issue, on='ad_id', how='left')

### Add AWS face federal

In [80]:
d['aws_face'] = d.apply(lambda row: ','.join(set(str(row['aws_face_img']).split(',')) | set(str(row['aws_face_vid']).split(','))) if (not pd.isnull(row['aws_face_img']) and not pd.isnull(row['aws_face_vid'])) else ','.join([item for item in str(row['aws_face_img']).split(',') + str(row['aws_face_vid']).split(',') if item != 'nan']) if pd.isnull(row['aws_face_img']) or pd.isnull(row['aws_face_vid']) else '', axis=1)

In [81]:
d['aws_face_federal'] = d['aws_face'].apply(lambda x: ','.join(list(set([item.strip() for item in str(x).split(',') if item.strip() in candlist]))))

In [82]:
d['aws_face_federal']= d['aws_face_federal'].replace('', np.NaN)

### Add combined entities federal

In [83]:
d['combined_entities_federal'] = (d['detected_entities_federal'].fillna('') + ',' + d['aws_face_federal'].fillna('')).str.strip(',')

In [84]:
d['combined_entities_federal']= d['combined_entities_federal'].replace('', np.NaN)

In [85]:
d['combined_entities_federal'] = d['combined_entities_federal'].apply(lambda x: list(set(x.split(','))) if isinstance(x, str) else np.nan)

In [86]:
d['combined_entities_federal'] = d['combined_entities_federal'].apply(lambda x: ','.join(x) if isinstance(x, list) else np.nan)

In [87]:
#d.loc[(d['detected_entities_federal'].notna()) &(d['aws_face_federal'].notna())]

### Add federal verified 

- text mention or image reference of federal candidates

In [88]:
d['federal_verified'] = d[['aws_face_federal','detected_entities_federal']].isna().sum(axis=1).apply(lambda x: 'No' if x==2 else 'Yes')

In [89]:
print(d.shape)
print(len(d.ad_id.unique()))

(377721, 64)
377721


In [90]:
d.federal_verified.value_counts(dropna=False)

federal_verified
No     261560
Yes    116161
Name: count, dtype: int64

In [91]:
d.columns

Index(['ad_id', 'page_id', 'ad_creation_time', 'ad_delivery_start_time',
       'ad_delivery_stop_time', 'spend', 'impressions', 'region_distribution',
       'demographic_distribution', 'publisher_platforms', 'potential_reach',
       'pd_id', 'wmp_media_type', 'aws_face_vid', 'aws_face_img', 'sub_bucket',
       'race_of_focus', 'race_of_focus_region_pct', 'ad_tone_constructed',
       'ad_tone_mentionbased', 'goal_DONATE_prediction',
       'goal_DONATE_predicted_prob', 'goal_CONTACT_prediction',
       'goal_CONTACT_predicted_prob', 'goal_PURCHASE_prediction',
       'goal_PURCHASE_predicted_prob', 'goal_GOTV_prediction',
       'goal_GOTV_predicted_prob', 'goal_EVENT_prediction',
       'goal_EVENT_predicted_prob', 'goal_POLL_prediction',
       'goal_POLL_predicted_prob', 'goal_GATHERINFO_prediction',
       'goal_GATHERINFO_predicted_prob', 'goal_LEARNMORE_prediction',
       'goal_LEARNMORE_predicted_prob', 'goal_PRIMARY_PERSUADE_prediction',
       'goal_PRIMARY_PERSUADE_predi

In [92]:
d.head()

,ad_id,page_id,ad_creation_time,ad_delivery_start_time,ad_delivery_stop_time,spend,impressions,region_distribution,demographic_distribution,publisher_platforms,...,attacklike_senti_prob,ABSA_number_of_mentions,ABSA_detected_entities,ABSA_predicted_sentiment_agg,issue_field,issue_class,aws_face,aws_face_federal,combined_entities_federal,federal_verified
0,x_1100321017564452,260713150211,2022-08-12,2022-08-13,2023-01-06,"{""lower_bound"":""100"",""upper_bound"":""199""}","{""lower_bound"":""2000"",""upper_bound"":""2999""}","[{""percentage"":""0.005223"",""region"":""Mississipp...","[{""percentage"":""0.005834"",""age"":""18-24"",""gende...","[""facebook"",""instagram""]",...,0.789673,NaN,NaN,NaN,page_name|ad_creative_link_title|disclaimer|ad...,ISSUE95|ISSUE95|ISSUE95|ISSUE80,,NaN,NaN,No
1,x_624349849305317,154561574598964,2022-08-29,2022-08-30,2023-01-08,"{""lower_bound"":""1500"",""upper_bound"":""1999""}","{""lower_bound"":""175000"",""upper_bound"":""199999""}","[{""percentage"":""1.0E-5"",""region"":""Comunidad Va...","[{""percentage"":""0.000698"",""age"":""25-34"",""gende...",facebook,...,0.818900,NaN,NaN,NaN,ad_creative_body|disclaimer|page_name|ad_creat...,"ISSUE12|ISSUE12,ISSUE80|ISSUE12,ISSUE80|ISSUE1...",WMPID5215,NaN,NaN,No
2,x_781950743004715,154561574598964,2022-08-29,2022-08-30,2023-01-07,"{""lower_bound"":""100"",""upper_bound"":""199""}","{""lower_bound"":""10000"",""upper_bound"":""14999""}","[{""percentage"":""0.000127"",""region"":""Washington...","[{""percentage"":""0.0014"",""age"":""45-54"",""gender""...",facebook,...,0.935922,NaN,NaN,NaN,disclaimer|ad_creative_body|page_name|ad_creat...,"ISSUE12,ISSUE80|ISSUE10,ISSUE12|ISSUE12,ISSUE8...",WMPID5215,NaN,NaN,No
3,x_469164518413893,154561574598964,2022-08-21,2022-08-30,2023-01-08,"{""lower_bound"":""1500"",""upper_bound"":""1999""}","{""lower_bound"":""200000"",""upper_bound"":""249999""}","[{""percentage"":""8.0E-6"",""region"":""Banteay Mean...","[{""percentage"":""0.002265"",""age"":""65+"",""gender""...",facebook,...,0.929551,NaN,NaN,NaN,disclaimer|ad_creative_body|page_name|ad_creat...,"ISSUE12,ISSUE80|ISSUE10,ISSUE12|ISSUE12,ISSUE8...",WMPID5215,NaN,NaN,No
4,x_708123550372840,103565195134756,2022-02-07,2022-02-07,2023-01-06,"{""lower_bound"":""8000"",""upper_bound"":""8999""}","{""lower_bound"":""1000000""}","[{""percentage"":""0"",""region"":""New Brunswick""},{...","[{""percentage"":""0.000131"",""age"":""18-24"",""gende...",facebook,...,0.973152,NaN,NaN,NaN,aws_ocr_text_vid,"ISSUE65,ISSUE95",,NaN,NaN,No


In [93]:
d.to_csv("../../fb_2022_adid_var.csv.gz", 
         index=False,
         compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})